In [3]:
# Connect Four Game - Blue Board, Fully Centered, Black Text

import numpy as np
import matplotlib.pyplot as plt
import random
from ipywidgets import HBox, VBox, Button, ToggleButtons, Output, Layout
from IPython.display import display, HTML

display(HTML("""
<style>
.widget-button, .widget-toggle-button {
    color: black !important;
    font-weight: bold !important;
}
</style>
"""))

ROWS, COLS = 6, 7

class ConnectFour:
    def __init__(self):
        self.board = np.zeros((ROWS, COLS), dtype=int)
        self.current = 1
        self.winner = 0
        self.moves = 0

    def drop(self, col):
        if self.winner or self.board[0, col] != 0:
            return False
        for r in range(ROWS-1, -1, -1):
            if self.board[r, col] == 0:
                self.board[r, col] = self.current
                self.moves += 1
                if self.check_win(r, col):
                    self.winner = self.current
                elif self.moves == ROWS*COLS:
                    self.winner = -1
                else:
                    self.current = 2 if self.current == 1 else 1
                return True
        return False

    def check_win(self, r, c):
        p = self.current
        for dr, dc in [(1,0),(0,1),(1,1),(1,-1)]:
            count = 1
            count += self.count(r, c, dr, dc, p)
            count += self.count(r, c, -dr, -dc, p)
            if count >= 4:
                return True
        return False

    def count(self, r, c, dr, dc, p):
        cnt = 0
        rr, cc = r+dr, c+dc
        while 0 <= rr < ROWS and 0 <= cc < COLS and self.board[rr, cc] == p:
            cnt += 1
            rr += dr
            cc += dc
        return cnt

    def reset(self, human_starts=True):
        self.board[:] = 0
        self.current = 1 if human_starts else 2
        self.winner = 0
        self.moves = 0

    def _get_next_empty_row(self, board, col):
        for r in range(ROWS-1, -1, -1):
            if board[r, col] == 0:
                return r
        return -1

    def _check_win_sim(self, board, r, c, p):
        for dr, dc in [(1,0),(0,1),(1,1),(1,-1)]:
            count = 1
            count += self._count_sim(board, r, c, dr, dc, p)
            count += self._count_sim(board, r, c, -dr, -dc, p)
            if count >= 4:
                return True
        return False

    def _count_sim(self, board, r, c, dr, dc, p):
        cnt = 0
        rr, cc = r + dr, c + dc
        while 0 <= rr < ROWS and 0 <= cc < COLS and board[rr, cc] == p:
            cnt += 1
            rr += dr
            cc += dc
        return cnt

game = ConnectFour()
out = Output()

def valid_columns():
    return [c for c in range(COLS) if game.board[0, c] == 0]

def ai_move():
    if game.winner:
        return
    vc = valid_columns()
    ai, human = 2, 1
    choice = -1

    if difficulty_toggle.value == 'Hard':
        for c in vc:
            r = game._get_next_empty_row(game.board, c)
            temp = game.board.copy()
            temp[r, c] = ai
            if game._check_win_sim(temp, r, c, ai):
                choice = c
                break
        if choice == -1:
            for c in vc:
                r = game._get_next_empty_row(game.board, c)
                temp = game.board.copy()
                temp[r, c] = human
                if game._check_win_sim(temp, r, c, human):
                    choice = c
                    break

    if choice == -1:
        choice = random.choice(vc)

    game.drop(choice)

def draw():
    with out:
        out.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(8,6))
        ax.set_xlim(-0.5, COLS - 0.5)
        ax.set_ylim(-0.5, ROWS - 0.5)
        ax.invert_yaxis()
        ax.set_xticks([])
        ax.set_yticks([])

        ax.set_facecolor('#0057b7')

        for r in range(ROWS):
            for c in range(COLS):
                color = 'black'
                if game.board[r, c] == 1:
                    color = 'red'
                elif game.board[r, c] == 2:
                    color = 'yellow'
                ax.add_patch(
                    plt.Circle((c, r), 0.42, color=color, ec='black', lw=1.8)
                )

        if game.winner == 1:
            title = "You Win!"
        elif game.winner == 2:
            title = "AI Wins!"
        elif game.winner == -1:
            title = "Draw!"
        else:
            title = "Your Turn (Red)" if game.current == 1 else "AI Turn (Yellow)"

        ax.set_title(title, fontsize=16, fontweight='bold', color='black')
        ax.set_aspect('equal')
        plt.show()

def on_col_click(col):
    if game.current != 1 or game.winner:
        return
    if game.drop(col):
        draw()
        if game.current == 2 and not game.winner:
            ai_move()
            draw()

col_colors = ['#ff9999','#ffcc99','#ffff99','#ccff99','#99ff99','#99ffcc','#99ccff']
col_buttons = []

for c in range(COLS):
    b = Button(
        description=f'Col {c+1}',
        layout=Layout(width='76px', height='45px'),
        style={'button_color': col_colors[c]}
    )
    b.on_click(lambda _, cc=c: on_col_click(cc))
    col_buttons.append(b)

start_toggle = ToggleButtons(
    options=['Human starts', 'AI starts'],
    value='Human starts',
    layout=Layout(padding='5px 10px')
)
difficulty_toggle = ToggleButtons(
    options=['Easy', 'Hard'],
    value='Easy',
    layout=Layout(padding='5px 10px')
)
reset_btn = Button(
    description='Reset',
    button_style='warning',
    layout=Layout(width='90px', height='27px')
)

def on_reset(_):
    game.reset(start_toggle.value == 'Human starts')
    draw()
    if game.current == 2:
        ai_move()
        draw()

reset_btn.on_click(on_reset)

controls_row = HBox(
    [start_toggle, difficulty_toggle, reset_btn],
    layout={'justify_content':'center','align_items':'center','gap':'12px'}
)

columns_row = HBox(
    col_buttons,
    layout={'justify_content':'center','align_items':'center', 'gap':'6px'}
)

board_row = HBox([out], layout={'justify_content':'center', 'align_items':'center'})

display(
    VBox([
        controls_row,
        columns_row,
        board_row
    ])
)

draw()